# Init

In [1]:
from data_preprocess import read_and_normalize_train_data, read_and_normalize_test_data2, save_submission
import numpy as np
import time
from vgg16_run import save_model, read_model

/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
img_rows, img_cols = 224, 224
batch_size = 32
nb_epoch = 10

In [3]:
import tensorflow as tf
from keras.utils import plot_model
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [4]:
def resnet50_model():
    base_model = ResNet50(include_top=False, input_shape=(224,224,3))
    resnet50_output = base_model.output
    predictions = Dense(10, activation='softmax')(Flatten()(resnet50_output))
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

In [5]:
from sklearn.model_selection import KFold

In [6]:
model_name='resnet50'
nfolds = 10
kf = KFold(n_splits=nfolds, shuffle=True, random_state=42)

# Train

In [7]:
x_train, y_train, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type=3)

Restore train from cache!


KeyboardInterrupt: 

In [8]:
x_train = np.transpose(x_train, [0,2,3,1])

In [9]:
print(x_train.shape, x_train.nbytes)
print(y_train.shape, y_train.nbytes)

(22424, 224, 224, 3) 13501759488
(22424, 10) 896960


In [10]:
num_fold = 0

for train_index, test_index in kf.split(x_train):
    num_fold += 1
    x = x_train[train_index]
    y = y_train[train_index]
    
    model = resnet50_model()
    model.fit(x, y, batch_size=batch_size, epochs=nb_epoch,
             verbose=1, validation_split=0.2, shuffle=True)
    
    save_model(model, num_fold, model_name)    

Train on 16144 samples, validate on 4037 samples
Epoch 1/10
16144/16144 [==============================] - 61s 4ms/step - loss: 1.2970 - val_loss: 8.4777
Epoch 2/10
16144/16144 [==============================] - 58s 4ms/step - loss: 0.7305 - val_loss: 9.3233
Epoch 3/10
16144/16144 [==============================] - 58s 4ms/step - loss: 0.5522 - val_loss: 10.3952
Epoch 4/10
16144/16144 [==============================] - 58s 4ms/step - loss: 0.4580 - val_loss: 10.2753
Epoch 5/10
16144/16144 [==============================] - 58s 4ms/step - loss: 0.3904 - val_loss: 10.5330
Epoch 6/10
16144/16144 [==============================] - 59s 4ms/step - loss: 0.3441 - val_loss: 11.2860
Epoch 7/10
16144/16144 [==============================] - 58s 4ms/step - loss: 0.3088 - val_loss: 11.0965
Epoch 8/10
16144/16144 [==============================] - 59s 4ms/step - loss: 0.2837 - val_loss: 11.7367
Epoch 9/10
16144/16144 [==============================] - 58s 4ms/step - loss: 0.2617 - val_loss: 11.3355

16145/16145 [==============================] - 63s 4ms/step - loss: 0.4544 - val_loss: 9.2893
Epoch 5/10
16145/16145 [==============================] - 63s 4ms/step - loss: 0.3896 - val_loss: 9.2205
Epoch 6/10
16145/16145 [==============================] - 63s 4ms/step - loss: 0.3452 - val_loss: 9.2623
Epoch 7/10
16145/16145 [==============================] - 63s 4ms/step - loss: 0.3142 - val_loss: 9.8142
Epoch 8/10
16145/16145 [==============================] - 63s 4ms/step - loss: 0.2836 - val_loss: 10.0353
Epoch 9/10
16145/16145 [==============================] - 63s 4ms/step - loss: 0.2621 - val_loss: 10.1639
Epoch 10/10
16145/16145 [==============================] - 63s 4ms/step - loss: 0.2427 - val_loss: 10.2998
Train on 16145 samples, validate on 4037 samples
Epoch 1/10
16145/16145 [==============================] - 68s 4ms/step - loss: 1.2792 - val_loss: 7.6076
Epoch 2/10
16145/16145 [==============================] - 60s 4ms/step - loss: 0.7342 - val_loss: 8.5648
Epoch 3/10
16

KeyboardInterrupt: 

In [13]:
del x
del y

NameError: name 'x' is not defined

# Train use ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('data/train', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 17943 images belonging to 10 classes.


In [8]:
validate_datagen = ImageDataGenerator(rescale=1./255)
validate_generator = train_datagen.flow_from_directory('data/validate', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 4481 images belonging to 10 classes.


In [9]:
model_name = 'resnet50'
model = resnet50_model()
model.fit_generator(train_generator, steps_per_epoch=2000, epochs=5, validation_data=validate_generator, validation_steps=800)
save_model(model, 100, model_name)

Epoch 1/5
2000/2000 [==============================] - 747s 373ms/step - loss: 0.3859 - val_loss: 4.8687
Epoch 2/5
2000/2000 [==============================] - 750s 375ms/step - loss: 0.0785 - val_loss: 5.8291
Epoch 3/5
2000/2000 [==============================] - 750s 375ms/step - loss: 0.0403 - val_loss: 6.6897
Epoch 4/5
2000/2000 [==============================] - 753s 377ms/step - loss: 0.0253 - val_loss: 7.3702
Epoch 5/5
2000/2000 [==============================] - 751s 376ms/step - loss: 0.0167 - val_loss: 7.6039


In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('data/test', target_size=(224,224), batch_size=batch_size)

Found 79726 images belonging to 1 classes.


['c0/img_10.jpg',
 'c0/img_100.jpg',
 'c0/img_1000.jpg',
 'c0/img_100000.jpg',
 'c0/img_100001.jpg',
 'c0/img_100002.jpg',
 'c0/img_100003.jpg',
 'c0/img_100004.jpg',
 'c0/img_100005.jpg']

In [15]:
test_ids = test_generator.filenames
test_ids = [d.strip('c0/') for d in test_ids]

In [17]:
model = read_model(100, model_name)
test_prediction = model.predict_generator(test_generator, verbose=1)
save_submission(test_ids, np.array(yfull_test))

2492/2492 [==============================] - 695s 279ms/step


In [12]:
test_paths = ['data/test/p'+str(i) for i in range(1,10)]

from vgg16_run import merge_several_folds_mean
print('Start testing............')

nfolds = 2
yfull_test = []
test_ids = []
for test_path in test_paths:
    test_data, test_id = read_and_normalize_test_data2(img_rows, img_cols, test_path, 3)
    print(test_data.shape)
    test_ids = test_ids + test_id
    
    test_data = np.transpose(test_data, (0,2,3,1))
    model = read_model(100, model_name)
    test_prediction = model.predict(test_data, batch_size=128, verbose=1)
    yfull_test = yfull_test + test_prediction.tolist()    
    

info_string = 'loss_' + model_name \
    + '_r_' + str(img_rows) \
    + '_c_' + str(img_cols) \
    + '_folds_' + str(100) \
    + '_ep_' + str(nb_epoch)

save_submission(test_ids, np.array(yfull_test))

  0%|          | 36/10381 [00:00<00:29, 352.75it/s]

Start testing............


 38%|███▊      | 3945/10381 [00:09<00:16, 395.68it/s]/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 10381/10381 [00:26<00:00, 396.94it/s]


Read test data time: 26.18 seconds
Directory doesn't exists
(10381, 3, 224, 224)
10381/10381 [==============================] - 35s 3ms/step


100%|██████████| 8666/8666 [00:21<00:00, 394.40it/s]


...

Read test data time: 21.65 seconds
Directory doesn't exists
(8659, 3, 224, 224)
8659/8659 [==============================] - 33s 4ms/step


100%|██████████| 8670/8670 [00:21<00:00, 399.47it/s]


Read test data time: 21.73 seconds
Directory doesn't exists
(8670, 3, 224, 224)
8670/8670 [==============================] - 34s 4ms/step


100%|██████████| 8708/8708 [00:22<00:00, 395.30it/s]


Read test data time: 22.05 seconds
Directory doesn't exists
(8708, 3, 224, 224)
8708/8708 [==============================] - 34s 4ms/step


# Test

In [7]:
test_paths = ['data/test/p'+str(i) for i in range(1,10)]

In [11]:
from vgg16_run import merge_several_folds_mean
print('Start testing............')

nfolds = 2
yfull_test = []
test_ids = []
for test_path in test_paths:
    test_data, test_id = read_and_normalize_test_data2(img_rows, img_cols, test_path, 3)
    print(test_data.shape)
    test_ids = test_ids + test_id
    y_test = []
    test_data = np.transpose(test_data, (0,2,3,1))
    for index in range(1, nfolds+1):
        # Store test predictions
        model = read_model(index, model_name)
        test_prediction = model.predict(test_data, batch_size=128, verbose=1)
        y_test.append(test_prediction)
    yfull_test = yfull_test + merge_several_folds_mean(y_test, nfolds)
    print('yfull_test.len:', len(yfull_test))

info_string = 'loss_' + model_name \
    + '_r_' + str(img_rows) \
    + '_c_' + str(img_cols) \
    + '_folds_' + str(nfolds) \
    + '_ep_' + str(nb_epoch)

save_submission(test_ids, yfull_test)

  0%|          | 36/10381 [00:00<00:28, 358.63it/s]

Start testing............


 38%|███▊      | 3954/10381 [00:09<00:16, 395.79it/s]/home/alpha/anaconda3/envs/tensorflow1.8/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 10381/10381 [00:26<00:00, 395.89it/s]


Read test data time: 26.25 seconds
Directory doesn't exists
(10381, 3, 224, 224)
10381/10381 [==============================] - 37s 4ms/step


  0%|          | 32/8666 [00:00<00:27, 313.98it/s]

...

  0%|          | 33/8708 [00:00<00:26, 323.38it/s]

yfull_test.len: 71018


100%|██████████| 8708/8708 [00:23<00:00, 365.99it/s]


Read test data time: 23.82 seconds
Directory doesn't exists
(8708, 3, 224, 224)
8708/8708 [==============================] - 42s 5ms/step
yfull_test.len: 79726


FileNotFoundError: [Errno 2] No such file or directory: 'result/submission_2018-05-05_22:09:05.csv'

In [14]:
yfull_test= np.array(yfull_test)
save_submission(test_ids, yfull_test)

In [37]:
import os
import numpy as np
import shutil
def split_train_data(train_path, validate_path, ratio=0.2):
    np.random.seed(42)
    for c in os.listdir(train_path):
        vpath = os.path.join(validate_path, c)
        tpath = os.path.join(train_path, c)
        os.mkdir(vpath)
        files = os.listdir(tpath)
        size = len(files)
        perm = np.random.permutation(size)
        selected = perm[:int(size*ratio)]
        for idx in selected:
            f = files[idx]
            tfile = os.path.join(tpath, f)
            shutil.move(tfile, vpath)
        print('classes {} done'.format(c))

In [38]:
split_train_data('data/train', 'data/validate')

classes c5 done
classes c8 done
classes c3 done
classes c1 done
classes c0 done
classes c9 done
classes c2 done
classes c7 done
classes c6 done
classes c4 done
